# Import Packages

In [1]:
!pip install -q streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 28.7 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.

In [2]:
import numpy as np
import pandas as pd
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load Data

In [3]:
raw_data_train = pd.read_csv('https://raw.githubusercontent.com/jmpark0808/pl_mnist_example/main/train_hp_msci436.csv')

Filter data for the features we're looking and one-hot encode the categorical variables


In [4]:
mid_df = raw_data_train[['SalePrice','YrSold','LotArea','MSSubClass','Neighborhood','YearBuilt','FullBath','HalfBath','BedroomAbvGr','GarageCars','KitchenQual','BldgType','CentralAir','GrLivArea']]
df = pd.get_dummies(data=mid_df, columns=['MSSubClass','Neighborhood','KitchenQual','BldgType','CentralAir'])

# Fit a Linear Regression Model

In [5]:
X = df.values[:, 1:]
y = df.values[:, 0]

In [6]:
reg = LinearRegression().fit(X, y)
reg.coef_



array([-3.17391218e+02,  4.97947931e-01,  5.08712532e+02,  5.96567839e+03,
        4.81692150e+03, -6.81445001e+03,  1.14098381e+04,  6.85345711e+01,
        9.05340821e+03,  7.44622227e+03,  9.98884026e+03,  9.01309355e+03,
       -1.79271423e+03, -1.18877199e+04,  4.16711899e+03,  9.33659203e+03,
        5.04941554e+03,  7.67461687e+03, -5.84608573e+03, -9.85246884e+03,
       -3.88192165e+04, -5.39107238e+03,  1.18599699e+04, -3.61099596e+03,
        1.02765809e+04,  1.17406112e+04, -8.42460507e+03, -5.39003527e+03,
       -1.06750276e+04,  1.77493108e+04, -2.72867383e+04, -1.88187171e+04,
       -2.20302557e+04, -7.54087362e+03, -1.81364916e+04, -1.79412226e+04,
        1.01797664e+04, -1.89974394e+04,  5.20396241e+04,  4.60727646e+04,
       -2.55453939e+04, -2.08028751e+04, -2.09532228e+04, -1.43873579e+04,
        1.00388107e+04,  6.02318045e+04, -6.32752532e+02,  2.28447310e+04,
        4.33438489e+04, -2.13924670e+04, -5.93211586e+03, -1.60192660e+04,
        1.30807896e+04,  

Calculate the Error

In [7]:
 mean_squared_error(X.dot(reg.coef_),y)

94930481034.17047

# Streamlit

Prepare the data for import into the model using pickling

In [8]:
pickle.dump(df, open('data','wb'))
pickle.dump(mid_df,open('raw','wb'))
pickle.dump(reg, open('regression','wb'))

In [9]:
%%writefile app.py
# Cheatsheet available at https://docs.streamlit.io/library/cheatsheet

# Import the relevant folters
import streamlit as st
import pickle as pk
import plotly.express as px
import numpy as np
import pandas as pd

# Load the data in the streamlit model
reg = pk.load(open('regression','rb'))
mid_df = pk.load(open('raw','rb'))
df = pk.load(open('data','rb'))

# Initialize array position offsets for easy accesss
MSSubClass_offset = 8
Neighbrhood_offset = 23
Kithcen_offset = 48
BldgType_offset = 52
CentralAir_offset = 57

# Sidebar for inputs
with st.sidebar:
  st.sidebar.header('Input Your Preferences!')
  st.write('Hello, Time to input your home price values :sunglasses:')

  YearBuilt = st.number_input("What year was the home built?", min_value=0, value=2000, step=1)

  YrSold = st.number_input('What year will the sale take place?', min_value=0, value=2008, step=1)

  LotArea = st.number_input("What is the lot size in square feet?", min_value=0, value=0, step=100)
  GrLivArea = st.number_input("What is the floor size above ground in square feet?", min_value=0, value=0, step=100)

  #Instead of saving each one-hot encoded variable separately, save the index offset of the chosen variable. Thus the chosen variable will have a value of 1, and all else will be 0
  MSSubClassOpts = ["STORY 1946 & NEWER ALL STYLES","1-STORY 1945 & OLDER","1-STORY W/FINISHED ATTIC ALL AGES","1-1/2 STORY - UNFINISHED ALL AGES","1-1/2 STORY FINISHED ALL AGES","2-STORY 1946 & NEWER","2-STORY 1945 & OLDER","2-1/2 STORY ALL AGES","SPLIT OR MULTI-LEVEL","SPLIT FOYER","DUPLEX - ALL STYLES AND AGES","1-STORY PUD (Planned Unit Development) - 1946 & NEWER","1-1/2 STORY PUD - ALL AGES","2-STORY PUD - 1946 & NEWER","PUD - MULTILEVEL - INCL SPLIT LEV/FOYER","2 FAMILY CONVERSION - ALL STYLES AND AGES"]
  MSSubClassTemp = st.selectbox('What is the building class?', MSSubClassOpts)
  MSSubClassIndex = MSSubClassOpts.index(MSSubClassTemp)

  BldgTypeOpts = list(np.sort(mid_df.loc[:,'BldgType'].replace('1Fam', 'Single-family Detached').replace('2fmCon','Two-family Conversion').replace('Duplx','Duplex').replace('TwnhsE', 'Townhouse End Unit').replace('TwnhsI','Townhouse Inside Unit').unique()))
  BldgTypeTemp = st.selectbox('What is the building type?', BldgTypeOpts)
  BldgTypeIndex = BldgTypeOpts.index(BldgTypeTemp)

  NeighborhoodOpts = list(np.sort(mid_df.loc[:,'Neighborhood'].replace('Blmngtn', 'Bloomington Heights').replace('Blueste', 'Bluestem').replace('BrDale', 'Briardale').replace('BrkSide', 'Brookside').replace('ClearCr', 'Clear Creek').replace('CollgCr', 'College Creek').replace('Crawfor', 'Crawford').replace('IDOTRR', 'Iowa DOT and Rail Road').replace('MeadowV', 'Meadow Village').replace('Mitchel', 'Mitchell').replace('NAmes', 'North Ames').replace('NoRidge', 'North Ridge').replace('NPkVill', 'Northpark Villa').replace('NridgHt', 'Northridge Heights').replace('NWAmes', 'Northwest Ames').replace('OldTown', 'Old Town').replace('SWISU', 'South & West of Iowa State University').replace('SawyerW', 'Sawyer West').replace('Somerst', 'Somerset').replace('StoneBr', 'Stone Brook').replace('Timber', 'Timberland').unique()))
  NeighborhoodTemp = st.selectbox('What is the neighbourhood?', NeighborhoodOpts)
  NeighborhoodIndex = NeighborhoodOpts.index(NeighborhoodTemp)

  FullBath = st.number_input('How many full bathrooms are there?', min_value=0, value=0)
  HalfBath = st.number_input('How many half bathrooms are there?', min_value=0, value=0)
  BedroomAbvGr  = st.number_input('How many bedroom above ground level are there?', min_value=0, value=0)
  GarageCars  =  st.number_input('How many standard cars can fit into the garage?', min_value=0, value=0)

  KitchenQualOpts = list(np.sort(mid_df.loc[:,'KitchenQual'].replace('Ex','Excellent').replace('Gd', 'Good').replace('TA', 'Typical/Average').replace('Fa','Fair').replace('Po','Poor').unique()))
  KitchenQualTemp = st.radio('What is the quality of the kitchen?', KitchenQualOpts)
  KitchenQualIndex = KitchenQualOpts.index(KitchenQualTemp)

  CentralAirOpts = list((mid_df.loc[:,'CentralAir'].replace('N', 'No').replace('Y', 'Yes').unique()))
  CentralAirTemp = st.radio('Is there central air conditioning?', CentralAirOpts)
  CentralAirIndex = CentralAirOpts.index(CentralAirTemp)

  #This variable is a vector representing the variables that will be multiplied together to get the final value
  input = [YrSold, LotArea,YearBuilt ,FullBath, HalfBath, BedroomAbvGr, GarageCars, GrLivArea] + [0]*51
  input[MSSubClassIndex + MSSubClass_offset] = 1
  input[NeighborhoodIndex + Neighbrhood_offset] = 1
  input[KitchenQualIndex + Kithcen_offset] = 1
  input[BldgTypeIndex + BldgType_offset] = 1
  input[CentralAirIndex + CentralAir_offset] = 1

pred = '{:,.0f}'.format(reg.coef_.dot(input)).replace(',',' ')

st.title('Real Estate Value Predictor')


st.markdown('### Your predicted price is: $' + pred)
st.write('To change the price, adjust the parameters on the left-hand sidebar.')

newdf = df.drop(labels=range(df.shape[0]),axis='index').drop(labels='SalePrice',axis='columns')

contributions = [float(reg.coef_[x]) * float(input[x]) for x in range(len(input))]

newdf.loc['0']= contributions

parameter_labels = ['Year Sold', 'Lot Area', 'Year Built', 'Full Baths', 'Half Baths',
          'Bedrooms Abvove Ground', 'Cars in Garage', 'Above Groudn Living Area',
          "1-STORY 1946 & NEWER ALL STYLES","1-STORY 1945 & OLDER","1-STORY W/FINISHED ATTIC ALL AGES",
          "1-1/2 STORY - UNFINISHED ALL AGES","1-1/2 STORY FINISHED ALL AGES","2-STORY 1946 & NEWER",
          "2-STORY 1945 & OLDER","2-1/2 STORY ALL AGES","SPLIT OR MULTI-LEVEL",
          "SPLIT FOYER","DUPLEX - ALL STYLES AND AGES","1-STORY PUD (Planned Unit Development) - 1946 & NEWER",
          "2-STORY PUD - 1946 & NEWER","PUD - MULTILEVEL - INCL SPLIT LEV/FOYER",
          "2 FAMILY CONVERSION - ALL STYLES AND AGES",
          "Bloomington Heights","Bluestem","Briardale","Brookside",
          "Clear Creek","College Creek","Crawford","Edwards","Gilbert",
          "Iowa DOT and Rail Road","Meadow Village","Mitchell",
          "North Ames","Northridge","Northpark Villa","Northridge Heights",
          "Northwest Ames","Old Town","South & West of Iowa State University",
          "Sawyer","Sawyer West","Somerset","Stone Brook","Timberland","Veenker",
          'Excellent Kitchen', 'Fair Kitchen', 'Good Kitchen', 'Typical Kitchen',
          'Single-family Detached', 'Two-family Conversion', 'Duplex', 'Townhouse End Unit',
          'Townhouse Inside Unit', 'Central Air', 'No Central Air']

d = {'Parameter': parameter_labels, 'Contribution to the Predicted Price': contributions , 'New House': ['new'] * 59}
mini_df = pd.DataFrame(data=d)

#Create the graph representing the contributions to the price
c = px.bar(mini_df, x="New House", y="Contribution to the Predicted Price",
             color='Parameter', barmode='relative',
             height=500, template='ggplot2', title='Contributions to the Price',
            hover_data={'New House':False, # remove species from hover data
                             'Parameter': True,
                             'Contribution to the Predicted Price' : True
                            }
             )
c.update_xaxes(showticklabels=False, visible = False)

st.plotly_chart(c)


#Create the sensitivity scatterplots
p = st.selectbox("Select the parameter chart you'd like to see a sensitivity scatterplot for: ", parameter_labels)
l = parameter_labels.index(p)
dis = {'Sale Price': df['SalePrice'], p: df.values[:,l+1]}
fig = px.scatter(dis, x=p, y='Sale Price', trendline="ols")
st.plotly_chart(fig)







Writing app.py


In [10]:
!streamlit run /content/app.py &>/content/logs.txt &

In [11]:
!curl ipv4.icanhazip.com
!npx localtunnel --port 8501

35.239.206.7
npx: installed 22 in 3.42s
your url is: https://spotty-years-eat.loca.lt
^C
